In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
GFDL = xr.open_mfdataset('data/NMME/GFDL-SPEAR/*.nc')
GFDL = GFDL.assign_coords(X=(((GFDL.X + 180) % 360) - 180)).sortby(['X'])

OSError: no files to open

In [ ]:
GFDL_eastern_east_africa = (GFDL.sel(Y=slice(-3.5, 8), X=slice(38, 50))
                               .rename(
    {'Y': 'latitude', 'X': 'longitude', 'prec': 'predicted precip', 'S': 'date of prediction', 'L': 'lead time'}))

# Interpolate and subset CHIRPS to match spatial resolution of GFDL
chirps_eastern_east_africa = (chirps.sel(latitude=slice(-3.5, 9), longitude=slice(37,
                                                                                  51))  # One degree of resolution must be added for interpolation
                              .interp_like(GFDL_eastern_east_africa, method='nearest'))

# Calculate realized dates for GFDL, convert to dataframe
GFDL_eastern_east_africa_df = GFDL_eastern_east_africa.to_dataframe().reset_index()
GFDL_eastern_east_africa_df['realization_time'] = GFDL_eastern_east_africa_df.apply(
    lambda x: x['date of prediction'] + pd.DateOffset(months=int(x['lead time'])), axis=1)

# Convert CHIRPS to dataframe
chirps_eastern_east_africa_df = chirps_eastern_east_africa.to_dataframe().reset_index()

# Merge CHIRPS and GFDL dataframe
eastern_east_africa_merged_df = GFDL_eastern_east_africa_df.merge(chirps_eastern_east_africa_df, how='left',
                                                                     left_on=['realization_time', 'latitude',
                                                                              'longitude'],
                                                                     right_on=['time', 'latitude',
                                                                               'longitude']).dropna()

# Save to .csv
eastern_east_africa_merged_df.to_csv('data/csv/eastern_east_africa_GFDL_merged.csv')

# Drop date of prediction, latitude, longitude
eastern_east_africa_merged_df = eastern_east_africa_merged_df.drop(
    columns=['date of prediction', 'latitude', 'longitude'])
# make a new column month that extracts month from realization time
eastern_east_africa_merged_df['month'] = eastern_east_africa_merged_df['realization_time'].dt.month

# Make calculations on the merged dataframe for plotting
corr_clean = eastern_east_africa_merged_df.drop(['realization_time', 'time'], axis=1).groupby(
    ['month', 'lead time', 'M']).corr(method='pearson').drop('precip', axis=1).droplevel(level=3).reset_index()
corr_clean_1 = corr_clean.drop(corr_clean.index[::2])

stat_eastern_east_africa = (eastern_east_africa_merged_df.drop(['realization_time', 'time'], axis=1)
                            .groupby(['month', 'lead time', 'M']).agg(['mean', 'std']).reset_index())
stat_eastern_east_africa.columns = stat_eastern_east_africa.columns.droplevel()
stat_eastern_east_africa.columns = ['month', 'lead time', 'M', 'pred mean', 'pred std', 'actual mean', 'actual std']
stat_eastern_east_africa_clean = stat_eastern_east_africa.merge(corr_clean_1.reset_index(),
                                                                on=['month', 'lead time', 'M'], how='left').drop(
    "index", axis=1)
stat_eastern_east_africa_clean.columns = ['month', 'lead time', 'M', 'pred mean', 'pred std', 'actual mean',
                                          'actual std', 'corr']
stat_eastern_east_africa_clean['conditional bias'] = stat_eastern_east_africa_clean.apply(
    lambda x: np.power(x['corr'] - x['pred std'] / x['actual std'], 2), axis=1)
stat_eastern_east_africa_clean['unconditional bias'] = stat_eastern_east_africa_clean.apply(
    lambda x: np.power((x['pred mean'] - x['actual mean']) / x['actual std'], 2), axis=1)
stat_eastern_east_africa_clean['potential skill'] = stat_eastern_east_africa_clean.apply(
    lambda x: np.power(x['corr'], 2), axis=1)
stat_eastern_east_africa_clean = stat_eastern_east_africa_clean.drop(
    ['corr', 'pred mean', 'pred std', 'actual mean', 'actual std'], axis=1)
stat_eastern_east_africa_clean['skill score'] = stat_eastern_east_africa_clean.apply(
    lambda x: x['potential skill'] - x['conditional bias'] - x['unconditional bias'], axis=1)
stat_eastern_east_africa_clean = stat_eastern_east_africa_clean.groupby(['month', 'lead time']).mean().drop('M',
                                                                                                            axis=1).reset_index()
stat_eastern_east_africa_clean['region'] = 'Eastern East Africa'